In [15]:
# Import necessary libraries
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to SQLite database (or create it)
conn = sqlite3.connect('school_platform.db')
cursor = conn.cursor()


In [16]:



# Load data from Excel file
excel_file = 'Latest TechTest-BI-Dataset 2021.xlsx'
teacher_activity = pd.read_excel(excel_file, sheet_name='Teacher Activity')
student_activity = pd.read_excel(excel_file, sheet_name='Student Activity')
data_dictionary = pd.read_excel(excel_file, sheet_name='Data Dictonary')



In [17]:

# Write data to SQLite tables
teacher_activity.to_sql('Teacher_Activity', conn, if_exists='replace', index=False)
student_activity.to_sql('Student_Activity', conn, if_exists='replace', index=False)




72516

In [18]:

# Top 5 schools with overall teachers’ login% > 60%
query1 = """
SELECT school_name, 
       AVG(CAST(is_present AS INT)) * 100 AS login_percentage
FROM Teacher_Activity
GROUP BY school_name
HAVING login_percentage > 60
ORDER BY login_percentage DESC
LIMIT 5;
"""
top_schools = pd.read_sql_query(query1, conn)
print("Top 5 Schools with Overall Teachers’ Login% > 60%:")
print(top_schools)


Top 5 Schools with Overall Teachers’ Login% > 60%:
Empty DataFrame
Columns: [school_name, login_percentage]
Index: []


In [24]:
plt.figure(figsize=(10, 6))
sns.barplot(x='login_percentage', y='school_name', data=top_schools_df, palette='viridis')
plt.title('Top 5 Schools with Highest Teacher Login Percentage')
plt.xlabel('Login Percentage')
plt.ylabel('School Name')
plt.show()

NameError: name 'top_schools_df' is not defined

<Figure size 1000x600 with 0 Axes>

In [19]:

# Teachers Login day over day change in percentage across all schools
query2 = """
SELECT school_name,
       DATE(record_date) AS login_date,
       (CAST(SUM(is_present) AS FLOAT) / COUNT(DISTINCT teacher_id)) * 100 AS login_percentage
FROM Teacher_Activity
GROUP BY school_name, login_date
ORDER BY school_name, login_date;
"""
day_over_day_change = pd.read_sql_query(query2, conn)
print("Teachers Login Day Over Day Change in Percentage Across All Schools:")
print(day_over_day_change)


Teachers Login Day Over Day Change in Percentage Across All Schools:
        school_name  login_date  login_percentage
0    ABU DHABI_1373  2020-10-25         15.384615
1    ABU DHABI_1373  2020-10-26         23.076923
2    ABU DHABI_1373  2020-10-27         15.384615
3    ABU DHABI_1373  2020-10-28         23.076923
4    ABU DHABI_1373  2020-10-29          0.000000
..              ...         ...               ...
191     AL AIN_1775  2020-11-03         11.764706
192     AL AIN_1775  2020-11-04         11.764706
193     AL AIN_1775  2020-11-05         17.647059
194     AL AIN_1775  2020-11-06          0.000000
195     AL AIN_1775  2020-11-07          0.000000

[196 rows x 3 columns]


In [26]:

# If each billable student pays $500/Month, what’s the revenue generated per school
query3 = """
WITH monthly_login AS (
SELECT school_name, 
       COUNT(DISTINCT student_id) * 500 AS revenue
FROM Student_Activity
WHERE is_present = 1
GROUP BY school_name ;
"""
revenue_per_school = pd.read_sql_query(query3, conn)
print("Revenue Generated per School:")
print(revenue_per_school)


DatabaseError: Execution failed on sql '
WITH monthly_login AS (
SELECT school_name, 
       COUNT(DISTINCT student_id) * 500 AS revenue
FROM Student_Activity
WHERE is_present = 1
GROUP BY school_name ;
': near ";": syntax error

In [21]:

# Find the number of teachers per school who logged in 3 consecutive days
query4 = """
SELECT school_name, 
       teacher_id, 
       COUNT(DISTINCT DATE(record_date)) AS consecutive_days_logged_in
FROM Teacher_Activity
WHERE is_present = 1
GROUP BY school_name, teacher_id
HAVING consecutive_days_logged_in >= 3;
"""
teachers_consecutive_logins = pd.read_sql_query(query4, conn)
print("Number of Teachers per School Who Logged in 3 Consecutive Days:")
print(teachers_consecutive_logins)


Number of Teachers per School Who Logged in 3 Consecutive Days:
       school_name  teacher_id  consecutive_days_logged_in
0   ABU DHABI_1373      300440                           7
1   ABU DHABI_1373      303432                           9
2   ABU DHABI_1373      306517                           9
3   ABU DHABI_1373      531571                           4
4   ABU DHABI_1498      556158                           8
..             ...         ...                         ...
73     AL AIN_1562      508232                           6
74     AL AIN_1562      508360                           5
75     AL AIN_1562      508392                          11
76     AL AIN_1775      573625                           5
77     AL AIN_1775      574009                           5

[78 rows x 3 columns]


In [22]:

# Find the weekly average for student login activity per school
query5 = """
SELECT school_name, 
       STRFTIME('%Y-%W', record_date) AS week_number,
       AVG(CAST(is_present AS INT)) * 100 AS login_percentage
FROM Student_Activity
GROUP BY school_name, week_number;
"""
weekly_average_logins = pd.read_sql_query(query5, conn)
print("Weekly Average for Student Login Activity per School:")
print(weekly_average_logins)


Weekly Average for Student Login Activity per School:
       school_name week_number  login_percentage
0   ABU DHABI_1373     2020-42         10.273973
1   ABU DHABI_1373     2020-43          7.261509
2   ABU DHABI_1373     2020-44         12.678288
3   ABU DHABI_1380     2020-42          0.696864
4   ABU DHABI_1380     2020-43          0.447984
5   ABU DHABI_1380     2020-44          0.754936
6   ABU DHABI_1476     2020-42          0.000000
7   ABU DHABI_1476     2020-43          0.000000
8   ABU DHABI_1476     2020-44          5.555556
9   ABU DHABI_1498     2020-42         32.142857
10  ABU DHABI_1498     2020-43         24.236818
11  ABU DHABI_1498     2020-44         28.456159
12  ABU DHABI_1530     2020-42         20.069204
13  ABU DHABI_1530     2020-43         17.525773
14  ABU DHABI_1530     2020-44         23.472302
15  ABU DHABI_1620     2020-42          1.418440
16  ABU DHABI_1620     2020-43          0.607903
17  ABU DHABI_1620     2020-44          0.118203
18  ABU DHABI_1

In [23]:

# Data visualization

# Top 5 Schools with Overall Teachers’ Login% > 60%
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='login_percentage', data=top_schools)
plt.title('Top 5 Schools with Overall Teachers’ Login% > 60%')
plt.xlabel('School ID')
plt.ylabel('Login Percentage')
plt.show()

# Teachers Login Day Over Day Change in Percentage Across All Schools
plt.figure(figsize=(12, 6))
for school in day_over_day_change['school_id'].unique():
    data = day_over_day_change[day_over_day_change['school_id'] == school]
    plt.plot(data['login_date'], data['day_over_day_change'], label=f'School {school}')
plt.title('Teachers Login Day Over Day Change in Percentage Across All Schools')
plt.xlabel('Date')
plt.ylabel('Day Over Day Change (%)')
plt.legend()
plt.show()

# Revenue Generated per School
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='revenue_per_school', data=revenue_per_school)
plt.title('Revenue Generated per School')
plt.xlabel('School ID')
plt.ylabel('Revenue ($)')
plt.show()

# Number of Teachers per School Who Logged in 3 Consecutive Days
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='teachers_with_3_consecutive_logins', data=teachers_consecutive_logins)
plt.title('Number of Teachers per School Who Logged in 3 Consecutive Days')
plt.xlabel('School ID')
plt.ylabel('Number of Teachers')
plt.show()

# Weekly Average for Student Login Activity per School
plt.figure(figsize=(10, 6))
sns.barplot(x='school_id', y='average_weekly_logins', data=weekly_average_logins)
plt.title('Weekly Average for Student Login Activity per School')
plt.xlabel('School ID')
plt.ylabel('Average Weekly Logins')
plt.show()

# Close the database connection
conn.close()


ValueError: Could not interpret input 'school_id'

<Figure size 1000x600 with 0 Axes>